In [1]:
import d2ltvm
import numpy as np
import tvm
from tvm import te

successd...



In [2]:
def broadcast_add(shape1, shape2):
    ''' broadcast add between two 2-dimensional tensors
        shape1, shape2 : the shapes of the input tensors
    '''
    assert len(shape1) == 2 and len(shape2) == 2, \
        "broadcast tensors should both be 2-dimension"
    for i in range(len(shape1)):
        assert shape1[i] == shape2[i] or shape1[i] == 1 or shape2[i] == 1, \
            "tensor shapes do not fit for broadcasting"
    A = te.placeholder(shape1, name='A')
    B = te.placeholder(shape2, name='B')
    m = shape1[0] if shape2[0] == 1 else shape2[0]
    n = shape1[1] if shape2[1] == 1 else shape2[1]
    f = lambda x, y: A[0 if shape1[0] == 1 else x, 0 if shape1[1] == 1 else y] + \
                B[0 if shape2[0] == 1 else x, 0 if shape2[1] == 1 else y]
    C = te.compute((m, n), f, name='C')
    return A, B, C

In [3]:
m = 3
n = 4
shape1 = (m, 1)
shape2 = (m, n)
A, B, C = broadcast_add(shape1, shape2)
s = te.create_schedule(C.op)
tvm.lower(s, [A, B, C]).show()

In [4]:
def get_broad_data(shape1, shape2, constructor=None):
    """ Return random tensors a, b
        and empty tensor c to store broadcast results between a and b
        shape1, shape2: shapes of input tensors
        constructor : user-defined tensor constructor
    """
    np.random.seed(0)
    a = np.random.normal(size=shape1).astype('float32')
    b = np.random.normal(size=shape2).astype('float32')
    out_shape = (shape1[0] if shape2[0] == 1 else shape2[0], 
                 shape1[1] if shape2[1] == 1 else shape2[1])
    c = np.empty(shape=out_shape).astype('float32')
    if constructor:
        a, b, c = [constructor(i) for i in (a, b, c)]
    return a, b, c

a, b, c = get_broad_data(shape1, shape2, tvm.nd.array)
mod = tvm.build(s, [A, B, C])
mod(a, b, c)
np.testing.assert_equal(a.asnumpy() + b.asnumpy(), c.asnumpy())

In [5]:
shape1 = (m, 1)
shape2 = (1, n)
A, B, C = broadcast_add(shape1, shape2)
s = te.create_schedule(C.op)
tvm.lower(s, [A, B, C]).show()

In [6]:
a, b, c = get_broad_data(shape1, shape2, tvm.nd.array)
mod = tvm.build(s, [A, B, C])
mod(a, b, c)
np.testing.assert_equal(a.asnumpy() + b.asnumpy(), c.asnumpy())
print(a.shape, b.shape, c.shape)

(3, 1) (1, 4) (3, 4)


# Summary
1.We can define a broadcast operator in TVM.

2.Broadcast be can performed along multiple dimensions